In [1]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, StackingRegressor
from sklearn.multioutput import MultiOutputRegressor
from random import random
from sklearn.model_selection import cross_validate
import matplotlib
from waveform_methods import binning
import pandas as pd

In [2]:
file = uproot.open('/home/andy/12360.root')
file2 = uproot.open('/home/andy/12362.root')

In [3]:
Xmax1 = file['MCPrimaryInfo']['ghMaxDepth'].array()
valueDepth = file['MCPrimaryInfo']['longNumCharged'].array()
Depth1 = file['MCPrimaryInfo']['longDepth'].array()
chi2_1 = file['CurvatureOnlyParams']['chi2_time'].array()
red1 = file['MCPrimaryInfo']['ghRedChiSqr'].array()
mass1 = [1 for i in range(len(red1))]

good = []
count = 0
for i in range(len(Xmax1)):
    if (red1[i]<300 and chi2_1[i]<5) and Xmax1[i]<900:
        good.append(1)
        count+=1
    else:
        good.append(0)
        
Xmax2 = file2['MCPrimaryInfo']['ghMaxDepth'].array()
valueDepth = file2['MCPrimaryInfo']['longNumCharged'].array()
Depth2 = file2['MCPrimaryInfo']['longDepth'].array()
chi2_2 = file2['CurvatureOnlyParams']['chi2_time'].array()
red2 = file2['MCPrimaryInfo']['ghRedChiSqr'].array()
mass2 = [4 for i in range(len(red2))]

good2 = []
for i in range(len(Xmax2)):
    if (red2[i]<300 and chi2_2[i]<5) and Xmax2[i]<650:
        good2.append(1)
        count+=1
    else:
        good2.append(0)
print(count)

172382


In [4]:
S125_1 = np.log10(file['LaputopParams']['s125'].array())
S125_2 = np.log10(file2['LaputopParams']['s125'].array())

In [5]:
A1 = file['CurvatureOnlyParams']['A'].array()
A2 = file2['CurvatureOnlyParams']['A'].array()
D1 = file['CurvatureOnlyParams']['D'].array()
D2 = file2['CurvatureOnlyParams']['D'].array()
N1 = file['CurvatureOnlyParams']['N'].array()
N2 = file2['CurvatureOnlyParams']['N'].array()
beta1 = file['LaputopParams']['beta'].array()
beta2 = file2['LaputopParams']['beta'].array()
zenith1 = file['Laputop']['zenith'].array()
zenith2 = file2['Laputop']['zenith'].array()
rlogl1 = file['CurvatureOnlyParams']['rlogl'].array()
nmini1 = file['CurvatureOnlyParams']['nmini'].array()
ndf1 = file['CurvatureOnlyParams']['ndf'].array()
llh1 = file['CurvatureOnlyParams']['llh'].array()
rlogl2 = file2['CurvatureOnlyParams']['rlogl'].array()
nmini2 = file2['CurvatureOnlyParams']['nmini'].array()
ndf2 = file2['CurvatureOnlyParams']['ndf'].array()
llh2 = file2['CurvatureOnlyParams']['llh'].array()

Xmax1_new = Xmax1
Xmax2_new = Xmax2

In [6]:
A = np.append(A1,A2)
D = np.append(D1,D2)
N = np.append(N1,N2)
S125 = np.append(S125_1,S125_2)
beta = np.log10(np.append(beta1,beta2))
chi2 = np.log10(np.append(chi2_1,chi2_2))
zenith = np.cos(np.append(zenith1,zenith2))
Xmax = np.append(Xmax1_new,Xmax2_new)
goodness = np.append(good,good2)
red = np.append(red1,red2)
mass = np.append(mass1,mass2)

In [7]:
input_variable = np.array([np.append(np.append(np.append(i,j),k),l) for i,j,k,l in zip(S125,A,zenith,chi2)])
output = list(zip(Xmax,goodness))

In [8]:
from sklearn.model_selection import train_test_split
seed = 7
test_size = 0.2
X_train, X_test, y_train_1, y_test_1 = train_test_split(input_variable,output , test_size=test_size, random_state=seed)

In [10]:
y_train = list(zip(*y_train_1))[1]
y_test = list(zip(*y_test_1))[1]
rng = np.random.RandomState(1)
regr_1 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=5,criterion='friedman_mse'),
                            n_estimators=3, random_state=rng,loss='square')
regr_1.fit(X_train,y_train)
    
y_1= regr_1.predict(X_test)
y_2 = regr_1.predict(X_train)

count_0 = 0
count_02 = 0
for value in y_train:
    if value==0:
        count_0+=1
for value in y_test:
    if value==0:
        count_02+=1
        
count = 0
count2 = 0
for value1,value2 in zip(y_1,y_test):
    if value1<0.5 and value2==0:
        count+=1

for value1,value2 in zip(y_2,y_train):
    if value1<0.5 and value2==0:
        count2+=1

print(count/count_02,count2/count_0,)

0.7318361955085865 0.7272928502914484


In [11]:
check = Xmax>300
check2 = Xmax<900
new_check= []
for i,j in zip(check,check2):
    if i and j:
        new_check.append(True)
    else:
        new_check.append(False)
    
mask = regr_1.predict(input_variable)
mask_convert_pre = []
for i in mask:
    if i>=0.5:
        mask_convert_pre.append(True)
    else:
        mask_convert_pre.append(False)

mask_convert = []
for i,j in zip(new_check,mask_convert_pre):
    if i and j:
        mask_convert.append(True)
    else:
        mask_convert.append(False)

In [12]:
print(np.corrcoef(np.array([A[mask_convert],
                            D[mask_convert],
                            N[mask_convert],
                            S125[mask_convert],
                            beta[mask_convert],
                            zenith[mask_convert],
                            Xmax[mask_convert],
                            chi2[mask_convert],
                            red[mask_convert],
                            mass[mask_convert],
                            goodness[mask_convert]])))

[[ 1.          0.10769763 -0.53568563  0.23258881  0.15365011  0.36393119
   0.2021192   0.51499543 -0.01496563 -0.16783158  0.07278479]
 [ 0.10769763  1.         -0.0547396  -0.54530852 -0.00119416 -0.21704836
   0.02970534  0.0157585  -0.06633587 -0.18580628  0.06644586]
 [-0.53568563 -0.0547396   1.          0.13077147 -0.02219754 -0.07379707
  -0.10090535 -0.49221396  0.04986321  0.1456864  -0.07579124]
 [ 0.23258881 -0.54530852  0.13077147  1.          0.14523713  0.48884709
   0.22834403 -0.0097806   0.12946698  0.06284922 -0.12252494]
 [ 0.15365011 -0.00119416 -0.02219754  0.14523713  1.          0.28431626
   0.19943719 -0.13933369 -0.00519244 -0.1774666   0.04933238]
 [ 0.36393119 -0.21704836 -0.07379707  0.48884709  0.28431626  1.
  -0.07182661 -0.0403996  -0.01568275  0.02571152  0.18291602]
 [ 0.2021192   0.02970534 -0.10090535  0.22834403  0.19943719 -0.07182661
   1.          0.0105383   0.05928502 -0.65710827 -0.12261836]
 [ 0.51499543  0.0157585  -0.49221396 -0.0097806 

In [13]:
mask_new = []
import random
for i,j in zip(mask_convert,mass):
    if (i and j==1) and random.random()<0.15:
        mask_new.append(False)
    else:
        mask_new.append(i)

In [24]:
input_variable2 = np.array([np.append(np.append(np.append(i,j),k),l) for i,j,k,l in zip(A[mask_new],D[mask_new],N[mask_new],beta[mask_new])])
output_new = mass[mask_new]

In [25]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(output_new.reshape(-1,1))

In [26]:
from sklearn.preprocessing import minmax_scale
input_variable_scaled = minmax_scale(input_variable2)

In [27]:
from sklearn.model_selection import train_test_split
seed = 7
test_size = 0.2
X_train, X_test, y_train_1, y_test_1 = train_test_split(input_variable2,onehot_encoded, test_size=test_size, random_state=seed)

In [28]:
import keras
from keras import initializers,regularizers
from keras.layers import Dense, Dropout, Flatten, Input, Concatenate, LeakyReLU
from keras.models import Model
import keras.backend as K
from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_math_ops as math_ops

In [30]:
input_layer = Input(shape=(4,))

model1 = Dense(7)(input_layer)

model1 = LeakyReLU()(model1)

model2 = Dense(4)(model1)

model2 = LeakyReLU()(model1)

model2 = Concatenate(axis=-1)([model1,model2])

predictions = Dense(2,activation='softmax')(model2)

model = Model(inputs=input_layer,outputs=predictions)

opt = keras.optimizers.RMSprop(decay=1e-5)
#opt= keras.optimizers.Adam(decay=1e-5,lr=3e-4)   

model.compile(optimizer=opt , loss = 'binary_crossentropy')

history = model.fit(X_train,y_train_1,
                    epochs=10,
                    validation_data = (X_test,y_test_1))


Train on 110354 samples, validate on 27589 samples
Epoch 1/10
110354/110354 [==============================] - 3s 26us/step - loss: 0.6964 - val_loss: 0.6861
Epoch 2/10
110354/110354 [==============================] - 3s 26us/step - loss: 0.6725 - val_loss: 0.6612
Epoch 3/10
110354/110354 [==============================] - 3s 25us/step - loss: 0.6686 - val_loss: 0.6670
Epoch 4/10
110354/110354 [==============================] - 3s 25us/step - loss: 0.6672 - val_loss: 0.6573
Epoch 5/10
110354/110354 [==============================] - 3s 26us/step - loss: 0.6647 - val_loss: 0.6662
Epoch 6/10
110354/110354 [==============================] - 3s 23us/step - loss: 0.6636 - val_loss: 0.6562
Epoch 7/10
110354/110354 [==============================] - 3s 24us/step - loss: 0.6617 - val_loss: 0.6571
Epoch 8/10
110354/110354 [==============================] - 3s 23us/step - loss: 0.6611 - val_loss: 0.6741
Epoch 9/10
110354/110354 [==============================] - 3s 24us/step - loss: 0.6606 - val

In [31]:
value = model.predict(X_test)
predicted = np.hstack(onehot_encoder.inverse_transform(value))
true = np.hstack(onehot_encoder.inverse_transform(y_test_1))

In [32]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(true,predicted))

[[9854 4158]
 [6166 7411]]
